## Transformations and Homogeneous Coordinates

### What are points at infinity? 

- In geometry, is is considered to be an ideal point or an idealizing limit used to depict the end of each line.

- In projective geometry, a pair of lines can be said to determine a  point similar to the fact that a pair of points can determine a line. It is said to be the point which is at the end of two parallel lines or basically the point which is supposed to be the intersection of these two parallels.      

- In homogeneous coordinates, it represents a point that gives us a sense of direction but is infinitely far away in that direction.


### What type of transformation can you apply to transform a point from infinity to a point that is not at infinity

- When we represent a vector in a homogeneous coordinate system, we add another dimension to the vector in the matrix representation. If we have a vector `<x, y, z>` in a 3 dimensional space, we can represent it as `[x', y', z', w]'` where w represents the added dimension. Now, `x = x'/w, y = y'/w and z = z'/w` are the corresponding relations. Therefore, if we have `w = 1`, we can easily map homogeneous coordinates to our original coordiante system.

- Wer can apply these homogeneous transformations to represent points at infinity. In this case, the value of w will be 0. And hence x, y, z all will be values obtained after dividing by zero leading to the tendency of infinity.

###  Find the vanishing point for the given images in the **data/Q3** folder. Complete function **FilterLines()** and  **GetVanishingPoint()** in the given starter code.

In [1]:
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

def ReadImage(InputImagePath):
    Images = []                     # Input Images will be stored in this list.
    ImageNames = []                 # Names of input images will be stored in this list.
    
    # Checking if path is of file or folder.
    if os.path.isfile(InputImagePath):						    # If path is of file.
        InputImage = cv2.imread(InputImagePath)                 # Reading the image.
        
        # Checking if image is read.
        if InputImage is None:
            print("Image not read. Provide a correct path")
            exit()
        
        Images.append(InputImage)                               # Storing the image.
        ImageNames.append(os.path.basename(InputImagePath))     # Storing the image's name.

	# If path is of a folder contaning images.
    elif os.path.isdir(InputImagePath):
		# Getting all image's name present inside the folder.
        for ImageName in os.listdir(InputImagePath):
			# Reading images one by one.
            InputImage = cv2.imread(InputImagePath + "/" + ImageName)
			
            Images.append(InputImage)							# Storing images.
            ImageNames.append(ImageName)                        # Storing image's names.
        
    # If it is neither file nor folder(Invalid Path).
    else:
        print("\nEnter valid Image Path.\n")
        exit()

    return Images, ImageNames
        
def GetLines(Image):
    # Converting to grayscale
    GrayImage = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
    # Blurring image to reduce noise.
    BlurGrayImage = cv2.GaussianBlur(GrayImage, (5, 5), 1)
    # Generating Edge image
    EdgeImage = cv2.Canny(BlurGrayImage, 40, 255)

    # Finding Lines in the image
    Lines = cv2.HoughLinesP(EdgeImage, 1, np.pi / 180, 50, 10, 15)

    # Check if lines found and exit if not.
    if Lines is None:
        print("Not enough lines found in the image for Vanishing Point detection.")
        exit(0)
    
    return Lines
    
def FilterLines(Lines):
    output = []
    maxLines = 100 # Maximum number of lines to be considered.
    critAngle = 5 # To Avoid Taking lines close to 0 or 90 degrees
    maxSlope = 1e5 # To represent Perpendicular lines.

    for everyLine in Lines:    
        # Getting the end points of the line.
        [x1, y1, x2, y2] = everyLine[0]   
        
        Slope = maxSlope
        if x1 != x2:
            Slope = (y2 - y1) / (x2 - x1)
            
        yIntercept = y2 - (Slope * x2)
        # y = mx + yIntercept
        
        angle = abs(math.degrees(math.atan(Slope)))
        if angle >= critAngle: # Avoid Horizontal lines.
            if angle <= 90 - critAngle: # Avoid Vertical lines.
                lengthLine = math.sqrt((y2 - y1) * (y2 - y1) + (x2 - x1) * (x2 - x1))
                output.append([Slope, yIntercept, lengthLine, x1, y1, x2, y2])
    
    # Sorting the lines based on length and taking the best few lines out of all.
    output = sorted(output, key=lambda x: x[2])
    if(len(output) > maxLines):
        output = output[:maxLines]

    return output 

def GetVanishingPoint(lines):
    err = 1e5 # To store minimum error across all points.
    ans = None 
    
    for i in range(len(lines)):
        for j in range(i + 1, len(lines)):
            slope1, intercept1 = lines[i][0], lines[i][1]
            slope2, intercept2 = lines[j][0], lines[j][1]
            if slope1 != slope2: # Avoiding parallel lines.
                pointError = 0
                xIntersect = (intercept2 - intercept1) / (slope1 - slope2)
                yIntersect = slope1 * xIntersect + intercept1
                for k in range(len(lines)): # Calculating error by iterating over all lines.
                    slope3, intercept3 = lines[k][0], lines[k][1]
                    perpSlope = -1 / slope3
                    newIntercept = yIntersect - perpSlope * xIntersect
                    xInt = (intercept3 - newIntercept) / (perpSlope - slope3) # X Intersection with perpendicular line.
                    yInt = perpSlope * xInt + newIntercept # Y Intersection with perpendicular line.
                    # Distance between the two points of intersection.
                    leng = math.sqrt((xInt - xIntersect) * (xInt - xIntersect)  + (yInt - yIntersect) * (yInt - yIntersect))
                    pointError += leng*leng
                pointError = math.sqrt(pointError)
                if pointError < err:  # Chek if new error is less than current minimum
                    err = pointError
                    ans = [xIntersect, yIntersect] # If error is minimum, the new point is the answer.
    return ans

Images, ImageNames = ReadImage("../../data/Q3/")            # Reading all input images


for i in range(len(Images)):
    Image = Images[i]

    # Getting the lines form the image
    Lines = GetLines(Image)

    FilteredLines = FilterLines(Lines)
    # Get vanishing point
    VanishingPoint = GetVanishingPoint(FilteredLines)

    # Checking if vanishing point found
    if VanishingPoint is None:
        print("Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.")
        continue

    # Drawing lines and vanishing point
    Lines = FilteredLines

    for Line in Lines:
        cv2.line(Image, (Line[3], Line[4]), (Line[5], Line[6]), (0, 255, 0), 2)
    cv2.circle(Image, (int(VanishingPoint[0]), int(VanishingPoint[1])), 10, (0, 0, 255), -1)

    # Showing the final image
    cv2.imshow("OutputImage", Image)
    cv2.waitKey(0)

###  Given *frame* transformation $A \rightarrow B$ ,  $B \rightarrow C$ ,  $D \rightarrow C$ ,  $A \rightarrow E$. Compute *frame transformation*  $D \rightarrow E$. Also, given the co-ordinates of a point *x* in *D's* frame, what transformation is required to get *x's*  co-ordinates in *E's* frame? 

Let us consider $P^{A}$ to be the position vector of $\vec P$ in frame A.

Let $T_{A}^{B}$ be the homogeneous transformation matrix of frame A relative to B. Hence, multiplying a vetor in frame B with this matrix would yield us the mapping of the point in frame A. Therefore, $P^{B}$ = $T_{A}^{B}P^{A}$ 


Therefore we are given: $T_{A}^{B}$, $T_{B}^{C}$, $T_{D}^{C}$ and $T_{A}^{E}$.

Now we want to obtain : $T_{D}^{E}$

We can use the statement: $P^{E}$ = $T_{D}^{E}P^{D}$ 

Now, $P^{E}$ = $T_{A}^{E}P^{A}$
Also, $P^{A}$ = $T_{B}^{A}P^{B}$. But we have $T_{A}^{B}$ which is nothing but the inverse of $T_{B}^{A}$. 

So, $T_{B}^{A}$ = $(T_{A}^{B})^{-1}$  
Therefore, Now, $P^{E}$ = $T_{A}^{E}P^{A}$ = $T_{A}^{E} (T_{A}^{B})^{-1} P^{B} $

Now, $P^{B}$ = $T_{C}^{B}P^{C}$
Also, $P^{C}$ = $T_{B}^{C}P^{B}$. But we have $T_{B}^{C}$ which is nothing but the inverse of $T_{C}^{B}$. 

So, $T_{C}^{B}$ = $(T_{B}^{C})^{-1}$  
Therefore, Now, $P^{E}$ = $T_{A}^{E} (T_{A}^{B})^{-1} P^{B} $ = $T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} P^{C} $

Now, $P^{C}$ = $T_{D}^{C}P^{D}$

Therefore, Now, $P^{E}$ = $T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} P^{C}$ = $T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} T_{D}^{C}P^{D} $

Now comparing with the original statement:

$T_{D}^{E} = T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} T_{D}^{C}$
